# Low-rank approximation on $\mathcal{P}(d)$ - the space of $d$-dimensional SPD matrices

In this notebook we want to get some intuition in different approaches for computing low-rank approximations for manifold-valued signals

In [72]:
using Manifolds
using Manopt
using LinearAlgebra
using NIfTI
using Plots

### Load data ###

In [73]:
# load data
# ni = niread("data/nifti_dt.nii.gz") * 1e9;
ni = niread("data/nifti_dt_nonlinear.nii.gz") * 1e9;

In [74]:
size(ni)

(112, 112, 50, 1, 6)

In [75]:
x = 49
y = 60
z = 25

ni[x,y,z,1,:]  # gets xx, yx, yy, zx, zy, zz

6-element Vector{Float64}:
  1.456667231281017
  0.08766011527772122
  1.8108553545559403
 -0.10191358174038712
 -0.15621183513392367
  1.4246920310156952

### Construct manifold ###

In [76]:
d1, d2, d3, _, d = size(ni)

(112, 112, 50, 1, 6)

In [77]:
# construct data as points on the manifold (and add 1e-5 * I for numerical stability)
predata = [
    [
    [ni[i,j,k,1,1] + 1e-5;; ni[i,j,k,1,2];; ni[i,j,k,1,4]]; 
    [ni[i,j,k,1,2];; ni[i,j,k,1,3] + 1e-5;; ni[i,j,k,1,5]]; 
    [ni[i,j,k,1,4];; ni[i,j,k,1,5];; ni[i,j,k,1,6] + 1e-5]
    ] for i=1:d1, j=1:d2, k=1:d3];

In [78]:
print(size(predata))

(112, 112, 50)

In [79]:
# pick a 2D slice 
# xlims [34,81] y: [32,79] z:[8,30]
# data = predata[34:81, 32:79, 8:30]
# data = predata[34:51,32:59,15] 
data = predata[34:81,32:79,15]
D1, D2 = size(data)

M = SymmetricPositiveDefinite(3)
println(size(data))
check_point.(Ref(M), data)

(48, 48)


48×48 Matrix{Nothing}:
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 ⋮                                   ⋱           ⋮                 
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing 

In [80]:
# Export slice image
asymptote_export_SPD("results/Camino2D_orig.asy", data=data);

In [81]:
q = 1. * Matrix(I, 3, 3)
#  construct data
log_q_data = log.(Ref(M), Ref(q), data);  # ∈ T_q P(3)^d1
exp_log_q_data = Symmetric.(exp.(Ref(M), Ref(q), log_q_data));
check_point.(Ref(M), exp_log_q_data)

48×48 Matrix{Nothing}:
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 ⋮                                   ⋱           ⋮                 
 nothing  nothing  nothing  nothing     nothing  nothing  nothing  nothing
 nothing  nothing  nothing  nothing  …  nothing  nothing  nothing  nothing
 nothing 

In [82]:
# check how transpose works
log_q_data[10,10]

3×3 Matrix{Float64}:
 -0.0878752   0.924822   0.301573
  0.924822   -0.442671  -0.263334
  0.301573   -0.263334  -1.30848

First do HOSVD without curvature reweighing

In [83]:
Gramm1  = Symmetric([sum(inner.(Ref(M), Ref(q), log_q_data[k,:,:], log_q_data[l,:,:])) for k=1:D1, l=1:D1]);
Gramm2  = Symmetric([sum(inner.(Ref(M), Ref(q), log_q_data[:,k,:], log_q_data[:,l,:])) for k=1:D2, l=1:D2]);

In [84]:
(_, U1) = eigen(Gramm1);
(_, U2) = eigen(Gramm2);

In [85]:
rank = 10
r1, r2= rank .* (1,1)
U1r = U1[:,end-r1+1:end]
U2r = U2[:,end-r2+1:end]


R_q_1 = Symmetric.([sum(U1r[:,i] .* log_q_data[:,j]) for i=1:r1, j=1:D2])
R_q = Symmetric.([sum(U2r[:,j] .* R_q_1[i,:]) for i=1:r1, j=1:r2]);


In [86]:
log_q_data_rr = Symmetric.([sum(U1r[i,:] .* R_q[:,j]) for i=1:D1, j=1:r2])
log_q_data_r = Symmetric.([sum(U2r[j,:] .* log_q_data_rr[i,:]) for i=1:D1, j=1:D2]);

In [87]:
log_q_data_r[1,1]

3×3 Symmetric{Float64, Matrix{Float64}}:
 -0.161939     -0.0202564  -0.000188603
 -0.0202564    -0.148514    0.0115468
 -0.000188603   0.0115468  -0.166189

In [88]:
# relative error
sqrt(sum(norm.(Ref(M), Ref(q), log_q_data_r - log_q_data) .^ 2))/ sqrt(sum(norm.(Ref(M), Ref(q), log_q_data) .^ 2))

0.6148818004471404

In [89]:
data_r = Symmetric.(exp.(Ref(M), Ref(q), Symmetric.(log_q_data_r)))
check_point.(Ref(M), data_r)
# manifold_error = sqrt(sum(distance.(Ref(M), data_r, data) .^2))
println(data_r[10,10])
println(data[10,10])

[0.6969281225245693 0.36825656099934584 0.15688127269562602; 0.36825656099934584 0.4810630216312708 0.07198329422830824; 0.15688127269562602 0.07198329422830824 0.16816243175780088]
[1.3084076124761421 0.8073706148081783 0.1168601604373265; 0.8073706148081783 0.9827463367520547 -0.05320955828724827; 0.1168601604373265 -0.05320955828724827 0.2935992917665939]


In [90]:
# Export compressed slice image
asymptote_export_SPD("results/Camino2D_low_rank.asy", data=data_r);

Next, let's see what happens if we choose the reweighted metric